In [1]:
# Init/Load model
from transformers import BartForConditionalGeneration, BartTokenizer
import numpy as np
import torch
import os

device = "cuda"

# Define a directory to save the models
SAVE_DIR = '../saved_models'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

start_epoch = 55

class SimpleBART(torch.nn.Module):
    def __init__(self):
        super(SimpleBART, self).__init__()
        if start_epoch > 0:
            self.bart = BartForConditionalGeneration.from_pretrained(os.path.join(SAVE_DIR, f'epoch_{start_epoch}'))
            print(f'Loaded epoch_{start_epoch}')
        else:
            self.bart = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
            print('Loaded facebook/bart-base')
        self.tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

    def forward(self, input_ids, attention_mask):
        return self.bart(input_ids=input_ids, attention_mask=attention_mask)


model = SimpleBART().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

Loaded epoch_55


In [2]:
# Load Raw Datasets
from torch.utils.data import Dataset, DataLoader
import csv

ACCUMULATION_STEPS = 14
BATCH_SIZE = 14 # best performing batch size so far (in execution performance)
DATA_SIZE = 0

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=200):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.pad_token_id = tokenizer.pad_token_id
        self.start_token_id = tokenizer.cls_token_id
        self.end_token_id = tokenizer.eos_token_id

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, tokens = self.data[idx]

        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=self.max_length)

        # Add start and end tokens and then pad
        tokens = [self.start_token_id] + tokens + [self.end_token_id]
        tokens_padded = [self.pad_token_id] * self.max_length
        tokens_padded[:len(tokens)] = tokens
        tokens_padded[len(tokens):] = [self.pad_token_id] * (self.max_length - len(tokens))

        return inputs["input_ids"].squeeze(0), inputs["attention_mask"].squeeze(0), torch.tensor(tokens_padded, dtype=torch.long)


def load_data_from_csv(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        data = [(row[0], [int(tok) for tok in row[1].split(",")]) for row in reader]

    return data

def apply_concept(params, validation=False):
    global validationLoader
    global dataloader
    global DATA_SIZE

    merged_data = load_data_from_csv(f"../concept/egg.csv")

    print("Concepts loaded;")
    for file_name, percentage in params.items():
        data = load_data_from_csv(f"../concept/{file_name}.csv")
        cutoff = int(len(data) * percentage)

        if validation:
            loaded_data = data[-cutoff:]
        else:
            loaded_data = data[:cutoff]

        print(f"    - {file_name}: {len(loaded_data)}")
        merged_data.extend(loaded_data)

    DATA_SIZE = len(merged_data)
    dataset = CustomDataset(merged_data, model.tokenizer)
    if validation:
        validationLoader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    else:
        dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    print(f'  Total: {DATA_SIZE}')

In [3]:
# Setup Validator
def validate():
    EOS_TOKEN_ID = model.tokenizer.eos_token_id

    model.eval()

    # Initialize counters for accuracy calculation
    total_correct_sequences = 0
    total_sequences = 0

    # Initialize counters for average sequence accuracy within the mask
    total_accuracy = 0

    with torch.no_grad():
        for batch_idx, (input_ids, attention_mask, targets) in enumerate(validationLoader):
            input_ids, attention_mask, targets = input_ids.to(device), attention_mask.to(device), targets.to(device)
            outputs = model(input_ids, attention_mask)
            logits = outputs.logits

            # Identify where the EOS token is in the target sequence
            eos_positions = (targets == EOS_TOKEN_ID).cumsum(dim=1).type(torch.bool)
            mask = ~eos_positions | (targets == EOS_TOKEN_ID)

            _, predicted = logits.max(2)
            correct_sequences = ((predicted == targets) | ~mask).all(dim=1).float().sum().item()
            total_sequences += targets.size(0)
            total_correct_sequences += correct_sequences

            # Compute the accuracy for each sequence
            correct_tokens_per_sequence = ((predicted == targets) & mask).float().sum(dim=1)
            total_tokens_per_sequence = mask.float().sum(dim=1)
            total_accuracy += (correct_tokens_per_sequence / total_tokens_per_sequence).sum().item()

    # Compute and print the accuracy for the entire validation dataset
    validation_accuracy = total_correct_sequences / total_sequences
    print(f"  Total Seq Acc: {validation_accuracy*100:.3f}%")
    avg_accuracy = total_accuracy / total_sequences
    print(f"    Avg Seq Acc: {avg_accuracy*100:.3f}%")

    return avg_accuracy, validation_accuracy

In [4]:
# Setup trainer
def save_model():
    global start_epoch
    model_save_path = os.path.join(SAVE_DIR, f'epoch_{start_epoch}')
    model.bart.save_pretrained(model_save_path)

def trainFor(num_epochs, target_loss=0, target_p50_loss=0, target_acc=1.0, target_seq_acc=1.0):
    global start_epoch

    EOS_TOKEN_ID = model.tokenizer.eos_token_id
    acc_batch = int(ACCUMULATION_STEPS / BATCH_SIZE)
    total_batches = len(dataloader)

    for epoch in range(start_epoch+1, start_epoch+num_epochs+1):
        start_epoch = epoch
        model.train()

        # Resetting the accumulated gradients
        optimizer.zero_grad()

        # Initialize counters for accuracy calculation
        total_correct_sequences = 0
        total_sequences = 0
        cumulative_loss = 0.0

        # Initialize list to store batch losses
        batch_losses = []

        for batch_idx, (input_ids, attention_mask, targets) in enumerate(dataloader):

            input_ids, attention_mask, targets = input_ids.to(device), attention_mask.to(device), targets.to(device)
            outputs = model(input_ids, attention_mask)
            logits = outputs.logits

            # Identify where the EOS token is in the target sequence
            eos_positions = (targets == EOS_TOKEN_ID).cumsum(dim=1).type(torch.bool)
            mask = ~eos_positions | (targets == EOS_TOKEN_ID)

            # Apply mask to filter out tokens after the EOS token for loss computation
            active_loss = mask.view(-1).bool()
            active_logits = logits.view(-1, logits.size(-1))[active_loss]
            active_labels = targets.view(-1)[active_loss]
            loss = criterion(active_logits, active_labels)

            _, predicted = logits.max(2)
            correct_sequences = ((predicted == targets) | ~mask).all(dim=1).float().sum().item()
            total_sequences += targets.size(0)
            total_correct_sequences += correct_sequences

            # Accumulate the gradients
            loss.backward()
            loss_val = loss.item()

            cumulative_loss += loss_val
            batch_losses.append(loss_val)

            isLast = batch_idx == len(dataloader) - 1

            # Only perform an optimization step every ACCUMULATION_STEPS
            if isLast or batch_idx % acc_batch == 0:
                optimizer.step()
                optimizer.zero_grad()

            print(f"\rEpoch: {epoch}, Batch: {batch_idx} of {total_batches}, loss: {loss_val:.6f}      ", end='')


        # Compute and print the accuracy for the entire epoch
        epoch_accuracy = total_correct_sequences / total_sequences
        cumulative_loss = cumulative_loss / len(batch_losses)
        p25_loss = np.percentile(batch_losses, 25)
        p50_loss = np.percentile(batch_losses, 50)
        p75_loss = np.percentile(batch_losses, 75)
        print(f"\rEpoch: {epoch}, Accuracy: {epoch_accuracy*100:.2f}%                                             ")
        print(f"  Loss: 25%: {p25_loss:.6f} 50%: {p50_loss:.6f} 75%: {p75_loss:.6f}\n   Avg: {cumulative_loss:.6f}")

        # Write loss values to loss.csv
        percentiles = [np.percentile(batch_losses, i) for i in range(101)]
        with open('loss.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([epoch] + percentiles)

        if epoch % 10 == 0: # Save the model
            save_model()

        seq_acc, total_acc = validate()

        if total_acc >= target_acc:
            break
        if cumulative_loss <= target_loss:
            break
        if p50_loss <= target_p50_loss:
            break

        if seq_acc >= target_seq_acc:
            break

    # Make sure last epoch is always saved
    save_model()

In [5]:
ACCUMULATION_STEPS = BATCH_SIZE # pure memorisation so accumulation won't help
apply_concept({"vocabulary": 1.0, "noise": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "noise": 0.02})
trainFor(50, target_seq_acc=0.50)
# (34mins)

Concepts loaded;
    - vocabulary: 6808
    - noise: 15614
  Total: 22423
Concepts loaded;
    - vocabulary: 6808
    - noise: 3122
  Total: 9931
Epoch: 1, Accuracy: 0.00%                                             
  Loss: 25%: 4.403065 50%: 4.849018 75%: 5.296900
   Avg: 4.866282
  Total Seq Acc: 0.004%
    Avg Seq Acc: 39.869%
Epoch: 2, Accuracy: 0.04%                                             
  Loss: 25%: 4.114165 50%: 4.543126 75%: 4.914825
   Avg: 4.488807
  Total Seq Acc: 0.000%
    Avg Seq Acc: 39.873%
Epoch: 3, Accuracy: 0.04%                                             
  Loss: 25%: 4.032996 50%: 4.430487 75%: 4.820580
   Avg: 4.396551
  Total Seq Acc: 0.076%
    Avg Seq Acc: 39.906%
Epoch: 4, Accuracy: 0.06%                                             
  Loss: 25%: 3.972137 50%: 4.378263 75%: 4.781254
   Avg: 4.345113
  Total Seq Acc: 0.120%
    Avg Seq Acc: 39.934%
Epoch: 5, Accuracy: 0.12%                                             
  Loss: 25%: 3.932351 50%: 4.367048

In [6]:
ACCUMULATION_STEPS = BATCH_SIZE # pure memorisation so accumulation won't help
apply_concept({"vocabulary": 1.0, "noise": 0.04})
trainFor(50, target_seq_acc=0.60)
# (5mins)

Concepts loaded;
    - vocabulary: 6808
    - noise: 6245
  Total: 13054
Epoch: 10, Accuracy: 13.81%                                             
  Loss: 25%: 2.656691 50%: 2.988907 75%: 3.271646
   Avg: 2.973416
  Total Seq Acc: 19.721%
    Avg Seq Acc: 68.222%


In [7]:
ACCUMULATION_STEPS = 28 # *14 close to 32
apply_concept({"vocabulary": 1.0, "noise": 0.06})
trainFor(50, target_seq_acc=0.65)
# (5mins)

Concepts loaded;
    - vocabulary: 6808
    - noise: 9368
  Total: 16177
Epoch: 11, Accuracy: 17.17%                                             
  Loss: 25%: 2.074187 50%: 2.373098 75%: 2.669538
   Avg: 2.376195
  Total Seq Acc: 27.387%
    Avg Seq Acc: 76.537%


Aiming for:
$$
\begin{align*}
  \frac{unique}{tokens} &= \frac{4174}{6808} = 61.31\%  & \text{sign pairs to text only used once} \\
  \frac{text}{tokens}   &= \frac{5342}{6808}  = 78.47\% & \text{sign pairs to text unique text} \\
  & & \text{unique meaning the text is only used for one tokenID}
\end{align*}
$$

i.e. there are six different signs which can be used for "present"  
which means we're actually aiming for $96\%$ effective accuracy $\frac{75\%}{78\%}$

But we don't want to over-fit either  
Hence why we slowly introduce new concepts while still memorising vocabulary

`target_seq_acc` includes the `EOS` token, which we of course we want to be right
So our target should be $\frac{61.31\% + 100\%}{2} = 80.65\%$

In [5]:
ACCUMULATION_STEPS = 70 # *14 close to 64
apply_concept({"vocabulary": 1.0, "noise": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "noise": 0.08})
trainFor(50, target_seq_acc=0.80655)
# (11mins)

Concepts loaded;
    - vocabulary: 6808
    - noise: 15614
  Total: 22423
Concepts loaded;
    - vocabulary: 6808
    - noise: 12491
  Total: 19300
Epoch: 12, Accuracy: 20.01%                                             
  Loss: 25%: 1.637285 50%: 1.861813 75%: 2.106627
   Avg: 1.887390
  Total Seq Acc: 32.244%
    Avg Seq Acc: 80.065%
Epoch: 13, Accuracy: 23.50%                                             
  Loss: 25%: 1.358196 50%: 1.585695 75%: 1.840750
   Avg: 1.606165
  Total Seq Acc: 34.451%
    Avg Seq Acc: 81.583%


The pure memorisation is over, as the vocabulary has been sufficiently learnt  
While it's not perfect, it will improve further over the later training, as the vocab remains in the training set

In [6]:
ACCUMULATION_STEPS = 1022 # *14 close to 1024
apply_concept({"vocabulary": 1.0, "noise": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "noise": 0.225})
trainFor(50, target_seq_acc=0.6)
# (10mins)

Concepts loaded;
    - vocabulary: 6808
    - noise: 15614
  Total: 22423
Concepts loaded;
    - vocabulary: 6808
    - noise: 35132
  Total: 41941
Epoch: 14, Accuracy: 18.78%                                             
  Loss: 25%: 1.329572 50%: 1.558345 75%: 1.814157
   Avg: 1.585541
  Total Seq Acc: 37.921%
    Avg Seq Acc: 83.464%


In [7]:
ACCUMULATION_STEPS = 1022 # *14 close to 1024
apply_concept({"vocabulary": 1.0, "noise": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "noise": 0.45})
trainFor(50, target_seq_acc=0.7)
# (16mins)

Concepts loaded;
    - vocabulary: 6808
    - noise: 15614
  Total: 22423
Concepts loaded;
    - vocabulary: 6808
    - noise: 70264
  Total: 77073
Epoch: 15, Accuracy: 20.15%                                             
  Loss: 25%: 1.079807 50%: 1.281758 75%: 1.525836
   Avg: 1.319893
  Total Seq Acc: 41.399%
    Avg Seq Acc: 85.418%


In [8]:
ACCUMULATION_STEPS = 1022 # *14 close to 1024
apply_concept({"vocabulary": 1.0, "noise": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "noise": 0.9})
trainFor(50, target_seq_acc=0.75)
# (30mins)

Concepts loaded;
    - vocabulary: 6808
    - noise: 15614
  Total: 22423
Concepts loaded;
    - vocabulary: 6808
    - noise: 140528
  Total: 147337
Epoch: 16, Accuracy: 23.13%                                             
  Loss: 25%: 0.838045 50%: 1.006275 75%: 1.225789
   Avg: 1.055101
  Total Seq Acc: 44.762%
    Avg Seq Acc: 87.021%


In [9]:
ACCUMULATION_STEPS = 1022 # *14 close to 1024
apply_concept({"vocabulary": 1.0, "noise": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "noise": 0.9})
trainFor(50, target_p50_loss=0.2, target_seq_acc=0.90)
# (533mins)

Concepts loaded;
    - vocabulary: 6808
    - noise: 15614
  Total: 22423
Concepts loaded;
    - vocabulary: 6808
    - noise: 140528
  Total: 147337
Epoch: 17, Accuracy: 26.92%                                             
  Loss: 25%: 0.662440 50%: 0.804522 75%: 0.992638
   Avg: 0.856268
  Total Seq Acc: 45.779%
    Avg Seq Acc: 87.521%
Epoch: 18, Accuracy: 29.49%                                             
  Loss: 25%: 0.562557 50%: 0.685241 75%: 0.851517
   Avg: 0.737888
  Total Seq Acc: 47.928%
    Avg Seq Acc: 88.370%
Epoch: 19, Accuracy: 31.52%                                             
  Loss: 25%: 0.491418 50%: 0.597656 75%: 0.744295
   Avg: 0.650201
  Total Seq Acc: 48.513%
    Avg Seq Acc: 88.522%
Epoch: 20, Accuracy: 32.73%                                             
  Loss: 25%: 0.447802 50%: 0.545643 75%: 0.681414
   Avg: 0.597653
  Total Seq Acc: 49.436%
    Avg Seq Acc: 88.904%
Epoch: 21, Accuracy: 34.06%                                             
  Loss: 25%: 0.41

In [10]:
apply_concept({"vocabulary": 1.0, "noise": 0.1}, validation=True)
validate()
apply_concept({"vocabulary": 1.0}, validation=True)
validate()
apply_concept({"noise": 0.1}, validation=True)
validate()

Concepts loaded;
    - vocabulary: 6808
    - noise: 15614
  Total: 22423
  Total Seq Acc: 52.165%
    Avg Seq Acc: 90.004%
Concepts loaded;
    - vocabulary: 6808
  Total: 6809
  Total Seq Acc: 77.486%
    Avg Seq Acc: 92.470%
Concepts loaded;
    - noise: 15614
  Total: 15615
  Total Seq Acc: 41.121%
    Avg Seq Acc: 88.925%


(0.88925186383812, 0.4112071725904579)

Now that the model has successfully applied a vocabulary swap we must start the grammatical transformation training.  
First we will start by just teaching it which words to ignore as they do not exist in Auslan

In [11]:
ACCUMULATION_STEPS = 4088 # *14 close to 4096
apply_concept({"vocabulary": 1.0, "word-masking": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "word-masking": 0.45})
trainFor(20, target_p50_loss=0.2, target_seq_acc=0.70)
# (57mins)

Concepts loaded;
    - vocabulary: 6808
    - word-masking: 10000
  Total: 16809
Concepts loaded;
    - vocabulary: 6808
    - word-masking: 45000
  Total: 51809
Epoch: 35, Accuracy: 9.80%                                             
  Loss: 25%: 5.300752 50%: 5.609476 75%: 5.946348
   Avg: 5.744106
  Total Seq Acc: 31.804%
    Avg Seq Acc: 58.800%
Epoch: 36, Accuracy: 9.81%                                             
  Loss: 25%: 4.243643 50%: 4.516359 75%: 4.793437
   Avg: 4.508092
  Total Seq Acc: 31.638%
    Avg Seq Acc: 64.020%
Epoch: 37, Accuracy: 9.89%                                             
  Loss: 25%: 3.599592 50%: 3.846125 75%: 4.072579
   Avg: 3.822467
  Total Seq Acc: 32.019%
    Avg Seq Acc: 64.472%
Epoch: 38, Accuracy: 10.03%                                             
  Loss: 25%: 3.236534 50%: 3.473460 75%: 3.700928
   Avg: 3.461110
  Total Seq Acc: 31.751%
    Avg Seq Acc: 65.270%
Epoch: 39, Accuracy: 10.03%                                             
  Loss: 

In [12]:
ACCUMULATION_STEPS = 4088 # *14 close to 4096
apply_concept({"vocabulary": 1.0, "word-masking": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "word-masking": 0.9})
trainFor(20, target_p50_loss=0.2, target_seq_acc=0.9)
# (mins)

Concepts loaded;
    - vocabulary: 6808
    - word-masking: 10000
  Total: 16809
Concepts loaded;
    - vocabulary: 6808
    - word-masking: 90000
  Total: 96809
Epoch: 40, Accuracy: 5.95%                                             
  Loss: 25%: 2.783838 50%: 3.019561 75%: 3.248209
   Avg: 3.012249
  Total Seq Acc: 31.828%
    Avg Seq Acc: 69.566%
Epoch: 41, Accuracy: 6.10%                                             
  Loss: 25%: 2.493118 50%: 2.705811 75%: 2.920518
   Avg: 2.705100
  Total Seq Acc: 32.019%
    Avg Seq Acc: 72.888%
Epoch: 42, Accuracy: 6.09%                                             
  Loss: 25%: 2.316473 50%: 2.525594 75%: 2.723895
   Avg: 2.516568
  Total Seq Acc: 32.310%
    Avg Seq Acc: 76.054%
Epoch: 43, Accuracy: 6.19%                                             
  Loss: 25%: 2.155381 50%: 2.360453 75%: 2.556274
   Avg: 2.357171
  Total Seq Acc: 32.013%
    Avg Seq Acc: 75.060%
Epoch: 44, Accuracy: 6.24%                                             
  Loss: 25

In [13]:
save_model()

Cancelled early because we don't want to put too much focus into just the word masking  
The model already has a strong grasp on the concept and can start learning word arranging

In [5]:
ACCUMULATION_STEPS = 8190 # *14 close to 8192
apply_concept({"vocabulary": 1.0, "word-pairs": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "word-pairs": 0.06})
trainFor(1, target_p50_loss=2.0, target_seq_acc=0.4)
# (4mins)

Concepts loaded;
    - vocabulary: 6808
    - word-pairs: 10400
  Total: 17209
Concepts loaded;
    - vocabulary: 6808
    - word-pairs: 6240
  Total: 13049
Epoch: 45, Accuracy: 57.50%                                             
  Loss: 25%: 0.770977 50%: 1.173722 75%: 1.598817
   Avg: 1.226389
  Total Seq Acc: 61.863%
    Avg Seq Acc: 85.398%


In [6]:
ACCUMULATION_STEPS = 8190 # *14 close to 8192
apply_concept({"vocabulary": 1.0, "word-pairs": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "word-pairs": 0.1})
trainFor(1, target_p50_loss=2.0, target_seq_acc=0.4)
# (5mins)

Concepts loaded;
    - vocabulary: 6808
    - word-pairs: 10400
  Total: 17209
Concepts loaded;
    - vocabulary: 6808
    - word-pairs: 10400
  Total: 17209
Epoch: 46, Accuracy: 52.90%                                             
  Loss: 25%: 0.827584 50%: 1.168207 75%: 1.525043
   Avg: 1.209725
  Total Seq Acc: 60.021%
    Avg Seq Acc: 85.148%


In [7]:
ACCUMULATION_STEPS = 8190 # *14 close to 8192
apply_concept({"vocabulary": 1.0, "word-pairs": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "word-pairs": 0.45})
trainFor(2, target_p50_loss=2.0, target_seq_acc=0.4)
# (12mins)

Concepts loaded;
    - vocabulary: 6808
    - word-pairs: 10400
  Total: 17209
Concepts loaded;
    - vocabulary: 6808
    - word-pairs: 46800
  Total: 53609
Epoch: 47, Accuracy: 39.09%                                             
  Loss: 25%: 0.956436 50%: 1.164306 75%: 1.400537
   Avg: 1.192730
  Total Seq Acc: 54.965%
    Avg Seq Acc: 83.826%


In [8]:
ACCUMULATION_STEPS = 8190 # *14 close to 8192
apply_concept({"vocabulary": 1.0, "grammar": 0.1, "word-pairs": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "grammar": 0.01, "word-pairs": 0.9})
trainFor(3, target_p50_loss=1.0, target_seq_acc=0.4)
# (75mins)

Concepts loaded;
    - vocabulary: 6808
    - grammar: 50000
    - word-pairs: 10400
  Total: 67209
Concepts loaded;
    - vocabulary: 6808
    - grammar: 5000
    - word-pairs: 93600
  Total: 105409
Epoch: 48, Accuracy: 35.26%                                             
  Loss: 25%: 1.029486 50%: 1.585286 75%: 3.445197
   Avg: 2.245614
  Total Seq Acc: 14.166%
    Avg Seq Acc: 35.970%
Epoch: 49, Accuracy: 38.52%                                             
  Loss: 25%: 0.895498 50%: 1.427730 75%: 3.029519
   Avg: 1.967145
  Total Seq Acc: 15.263%
    Avg Seq Acc: 37.225%
Epoch: 50, Accuracy: 40.25%                                             
  Loss: 25%: 0.770915 50%: 1.255461 75%: 2.870213
   Avg: 1.825772
  Total Seq Acc: 15.060%
    Avg Seq Acc: 37.681%


In [5]:
ACCUMULATION_STEPS = 8190 # *14 close to 8192
apply_concept({"vocabulary": 1.0, "grammar": 0.1, "word-pairs": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "grammar": 0.9, "word-pairs": 0.9})
trainFor(5, target_p50_loss=1.0, target_seq_acc=0.3)
# (114mins)

Concepts loaded;
    - vocabulary: 6808
    - grammar: 50000
    - word-pairs: 10400
  Total: 67209
Concepts loaded;
    - vocabulary: 6808
    - grammar: 450000
    - word-pairs: 93600
  Total: 550409
Epoch: 51, Accuracy: 6.26%                                             
  Loss: 25%: 4.416064 50%: 4.600059 75%: 4.796405
   Avg: 4.610680
  Total Seq Acc: 12.513%
    Avg Seq Acc: 39.037%


In [6]:
ACCUMULATION_STEPS = 8190 # *14 close to 8192
apply_concept({"vocabulary": 1.0, "grammar": 0.1, "word-pairs": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "grammar": 0.9, "word-pairs": 0.9})
trainFor(3, target_p50_loss=1.5, target_seq_acc=0.8)
# (114mins)

Concepts loaded;
    - vocabulary: 6808
    - grammar: 50000
    - word-pairs: 10400
  Total: 67209
Concepts loaded;
    - vocabulary: 6808
    - grammar: 450000
    - word-pairs: 93600
  Total: 550409
Epoch: 52, Accuracy: 5.56%                                             
  Loss: 25%: 4.032571 50%: 4.196765 75%: 4.365591
   Avg: 4.200637
  Total Seq Acc: 11.856%
    Avg Seq Acc: 40.983%


In [7]:
ACCUMULATION_STEPS = 8190 # *14 close to 8192
apply_concept({"vocabulary": 1.0, "grammar": 0.1, "word-pairs": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "grammar": 0.9, "word-pairs": 0.9})
trainFor(3, target_p50_loss=1.5, target_seq_acc=0.8)
# (350mins)

Concepts loaded;
    - vocabulary: 6808
    - grammar: 50000
    - word-pairs: 10400
  Total: 67209
Concepts loaded;
    - vocabulary: 6808
    - grammar: 450000
    - word-pairs: 93600
  Total: 550409
Epoch: 53, Accuracy: 6.00%                                             
  Loss: 25%: 3.714889 50%: 3.890947 75%: 4.075457
   Avg: 3.898209
  Total Seq Acc: 11.595%
    Avg Seq Acc: 43.582%
Epoch: 54, Accuracy: 6.33%                                             
  Loss: 25%: 3.452131 50%: 3.632052 75%: 3.821001
   Avg: 3.641159
  Total Seq Acc: 12.750%
    Avg Seq Acc: 46.063%
Epoch: 55, Accuracy: 6.77%                                             
  Loss: 25%: 3.258086 50%: 3.442890 75%: 3.641506
   Avg: 3.452856
  Total Seq Acc: 13.349%
    Avg Seq Acc: 48.018%


In [5]:
ACCUMULATION_STEPS = 8190 # *14 close to 8192
apply_concept({"vocabulary": 1.0, "grammar": 0.1, "word-pairs": 0.1}, validation=True)
apply_concept({"vocabulary": 1.0, "grammar": 0.9, "word-pairs": 0.9})
trainFor(5, target_p50_loss=1.5, target_seq_acc=0.8)
# (567mins)

Concepts loaded;
    - vocabulary: 6808
    - grammar: 50000
    - word-pairs: 10400
  Total: 67209
Concepts loaded;
    - vocabulary: 6808
    - grammar: 450000
    - word-pairs: 93600
  Total: 550409
Epoch: 56, Accuracy: 7.02%                                             
  Loss: 25%: 3.151639 50%: 3.343688 75%: 3.546811
   Avg: 3.354264
  Total Seq Acc: 13.440%
    Avg Seq Acc: 48.859%
Epoch: 57, Accuracy: 7.16%                                             
  Loss: 25%: 3.025592 50%: 3.215463 75%: 3.418622
   Avg: 3.228399
  Total Seq Acc: 13.675%
    Avg Seq Acc: 50.155%
Epoch: 58, Accuracy: 7.30%                                             
  Loss: 25%: 2.901687 50%: 3.097794 75%: 3.305452
   Avg: 3.109050
  Total Seq Acc: 13.959%
    Avg Seq Acc: 51.758%
Epoch: 59, Accuracy: 7.53%                                             
  Loss: 25%: 2.777439 50%: 2.974927 75%: 3.184613
   Avg: 2.986968
  Total Seq Acc: 14.226%
    Avg Seq Acc: 53.113%
Epoch: 60, Accuracy: 7.76%                